'''
Task4 传统机器学习 

1. 朴素贝叶斯的原理
2. 利用朴素贝叶斯模型进行文本分类
3. SVM的原理
4. 利用SVM模型进行文本分类
5. pLSA、共轭先验分布；LDA主题模型原理
6. 使用LDA生成主题特征，在之前特征的基础上加入主题特征进行文本分类
7. 参考
朴素贝叶斯1：https://blog.csdn.net/u013710265/article/details/72780520
LDA数学八卦
lda2：https://blog.csdn.net/u013710265/article/details/73480332
合并特征：https://blog.csdn.net/u013710265/article/details/72848564

'''

In [2]:
import pandas as pd
import jieba

In [ ]:
'''
分类任务数据准备
'''
#==1、准备数据==
'''

dicts = {
    'title':['公安机关销毁１０余万非法枪支跨国武器走私渐起','张绍刚发道歉信网友不认可：他的问题是俯视他人（图）','欧洲杯大战在即荷兰葡萄牙面临淘汰将背水一战','国资委回应央企审计报告称９７％问题完成修改','金正恩为朝少年团代表安排宴会 学生发誓坚决跟随（图）','跳水运动员','赵丽颖生子'],
    'type':['军事','娱乐','体育','政治','政治','体育','娱乐'],
    'content':['中广网唐山６月１２日消息（记者汤一亮　庄胜春）据中国之声《新闻晚高峰》报道，今天（１２日）上午，公安机关２０１２年缉枪制爆专项行动“统一销毁非法枪爆物品活动”在河北唐山正式启动，１０万余只非法枪支、２５０余吨炸药在全国１５０个城市被统一销毁。黄明：现在我宣布，全国缉枪制爆统一销毁行动开始！随着公安部副部长黄明一声令下，大量仿制式枪以及猎枪、火药枪、气枪在河北唐山钢铁厂被投入炼钢炉。与此同时，在全国各省区市１５０个城市，破案追缴和群众主动上缴的１０万余支非法枪支被集中销毁，在全国各指定场所，２５０余吨炸药被分别销毁。公安部治安局局长刘绍武介绍，这次销毁的非法枪支来源于三个方面。刘绍武：打击破案包括涉黑、涉恶的团伙犯罪、毒品犯罪','天津卫视求职节目《非你莫属》“晕倒门”事件余波未了，主持人张绍刚前日通过《非你莫属》节目组发出道歉信，称自己错在对留学生缺乏了解。但他的道歉，没有得到网友的接受和原谅，有网友尖锐指出，张绍刚的问题就在俯视他人，连道歉都不会。张绍刚：我是一番好意之前哪怕被网友骂得再凶，张绍刚也表现彪悍，声称自己没错，绝不道歉但这几天李开复领衔讨伐节目组，网上民意汹汹要他“下课”。','中广网北京６月１３日消息（记者王宇）据中国之声《新闻晚高峰》报道，明天凌晨两场欧洲杯的精彩比赛上演，死亡之组Ｂ组当中两支传统的强队荷兰队和葡萄牙队正面临着提前淘汰出局的危险。第一轮之后Ｂ组的积分的形势是３、３、０、０，德国和丹麦３分','中新社北京６月２日电（记者　刘辰瑶）国资委２日在官网上回应了日前国家审计署发布的１５家中央企业２０１０年财务收支等审计结果公告，表示欢迎社会对央企进行监督。被“点名”的央企也于２日前纷纷对审计署提出的问题进行“检讨”，并称将进一步完善其制度。据悉，审计署１日公布了中石油、中石化','中新网６月８日电　据朝中社报道，２６０多名朝鲜少年团代表７日参加了朝鲜劳动党第一书记、朝鲜国防委员会第一委员长金正恩为他们自安排的宴会。金正恩为少年团代表们送来牛肉、虹鳟鱼等，给予他们悉心的照顾。少年团代表们发誓，坚决只相信并跟随金正恩，准备成为强盛国家坚定的骨干。　（来源：中国新闻网）,政治','郭晶晶取得四次奥运会金牌，田亮退役后参加了娱乐综艺节目爸爸去哪','今天下午，冯绍峰宣布赵丽颖于今日产下一名小王子，恭喜恭喜！据悉，2019年1月1日，冯绍峰发文正式宣布赵丽颖怀孕喜讯，“知否知否，应是一家三口”。女方随后转发写道“等候等候 恭迎2019”。赵丽颖冯绍峰于去年女方生日10月16日10时07分“官宣”结婚。']
}
df = pd.DataFrame(dicts)
df.head()
'''

In [6]:
#==用day2下载的数据（用上面7条数据训练结果是0分，所以数据太少也不行）==
with open(r'cnews.txt','r',encoding='utf-8') as fnews:
    newsLines = fnews.readlines()
#print(newsLines)
catergory = []
contentN = []
for newsItem in newsLines:
    catergory.append(newsItem.split('\t')[0])#\t以tab符切分
    contentN.append(newsItem.split('\t')[1])

dictNews = {'type':catergory,'content':contentN}
    
df = pd.DataFrame(dictNews)
df.head()

,type,content
0,体育,湖人新秀客场酒店醉酒闹事 警局一游错过黄蜂之战新浪体育讯北京时间4月26日消息，据雅虎体育报...
1,体育,魔鬼训练场爬出真奇兵 邓华德：他值得所有人尊敬华奥星空上海2月16日电(记者 李旭)“彭飞就...
2,体育,灰熊若黑八合情又合理！ 马刺天敌实力被大大低估新浪体育讯在刚刚结束的比赛中，西部第一马刺在客...
3,体育,皇帝关键球26%准度强于韦德 用他最后一投不是乱来新浪体育讯北京时间4月26日，再强大的球队...
4,体育,科比拒绝接受进一步检查 三方式治疗第五场一定上新浪体育讯北京时间4月26日消息(洛杉矶时间4...


In [7]:
#==2、分词、去停用词==

#读停用词表
with open(r'cnews.vocab.txt','r',encoding='utf-8') as fStop:
    stopWords = fStop.readlines()
#print(stopWords)
stopWordsList = []
for sw in stopWords:
    stopWordsList.append(sw.split('\n')[0])

#取文章内容
#print(stopWordsList)
lines = df.content.values#.tolist()可以不用加，values取出就是list格式了
#print(lines)#看下格式，jieba切分的文本需要list格式
#开始分词
segList = []
for line in lines:
    contentSeg = jieba.lcut(line,cut_all=False)#精确模式
    #print(segment) 
    if len(contentSeg) > 1 and contentSeg !='\r\n':
        #【注意此处是关键，以回车符为界按文章分组存回，而不是整个分好词以后所有文章分词存做list的一个元素】       contentSeg.append(seg_list)
        #【为什么不按文章分？因为先分好了以后才能找到\n换行符】
        #wordsList = [wd for wd in seg_list if wd not in stopWordsList]
        segList.append(contentSeg)
#print("==分词：==\n",segList)

#组装成dataFrame看下分好词的数据
dfSeg = pd.DataFrame({'segList':segList})#DataFrame传参字典即可组装完成
print("\n\n==每篇文章初次分词 DataFrame数据表展示==")
dfSeg.head()


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Administrator\AppData\Local\Temp\jieba.cache
Loading model cost 1.676 seconds.
Prefix dict has been built succesfully.




==每篇文章初次分词 DataFrame数据表展示==


,segList
0,"[湖人, 新秀, 客场, 酒店, 醉酒, 闹事, , 警局, 一游, 错过, 黄蜂, 之战..."
1,"[魔鬼, 训练场, 爬, 出, 真, 奇兵, , 邓华德, ：, 他, 值得, 所有人, ..."
2,"[灰熊, 若, 黑八, 合情, 又, 合理, ！, , 马刺, 天敌, 实力, 被, 大大..."
3,"[皇帝, 关键球, 26%, 准度, 强于, 韦德, , 用, 他, 最后, 一投, 不是..."
4,"[科比, 拒绝接受, 进一步, 检查, , 三, 方式, 治疗, 第五场, 一定, 上, ..."


In [327]:
#去停用词
wordsClean = []
for seg in segList:
    segClean = []
    for word in seg:
        if len(word) >0 and word not in stopWordsList:
            segClean.append(word)
    wordsClean.append(segClean)
#print(wordsClean)

#组装成DataFrame格式看效果
dfSeg_clean = pd.DataFrame({'wordsClean':wordsClean})
print("===去停用词后===")
dfSeg_clean.head()      

===去停用词后===


,wordsClean
0,"[湖人, 新秀, 客场, 酒店, 醉酒, 闹事, 警局, 一游, 错过, 黄蜂, 之战, 新..."
1,"[魔鬼, 训练场, 奇兵, 邓华德, 值得, 所有人, 尊敬, 华奥, 星空, 上海, 16..."
2,"[灰熊, 黑八, 合情, 合理, 马刺, 天敌, 实力, 大大, 低估, 新浪, 体育讯, ..."
3,"[皇帝, 关键球, 26%, 准度, 强于, 韦德, 最后, 一投, 不是, 乱来, 新浪,..."
4,"[科比, 拒绝接受, 进一步, 检查, 方式, 治疗, 第五场, 一定, 新浪, 体育讯, ..."


In [331]:
#==用贝叶斯 训练数据 分类 验证模型==

# ==3、组装数据==
#把上面分好词的词袋与label（最初文章所属类别列）组装起来
dfDatas = pd.DataFrame({'features':wordsClean,'label':df['type']})
dfDatas.head()


,features,label
0,"[湖人, 新秀, 客场, 酒店, 醉酒, 闹事, 警局, 一游, 错过, 黄蜂, 之战, 新...",体育
1,"[魔鬼, 训练场, 奇兵, 邓华德, 值得, 所有人, 尊敬, 华奥, 星空, 上海, 16...",体育
2,"[灰熊, 黑八, 合情, 合理, 马刺, 天敌, 实力, 大大, 低估, 新浪, 体育讯, ...",体育
3,"[皇帝, 关键球, 26%, 准度, 强于, 韦德, 最后, 一投, 不是, 乱来, 新浪,...",体育
4,"[科比, 拒绝接受, 进一步, 检查, 方式, 治疗, 第五场, 一定, 新浪, 体育讯, ...",体育


In [332]:
#==4、label向量化==

#首先把分类做一个到数字的映射
print(dfDatas['label'].unique())
#看下不重复的分类类别
#['体育' '娱乐' '家居' '房产' '教育' '时尚' '时政' '游戏' '科技' '财经']

#做一个汉字分类到数字向量的字典映射
labelDict = {'体育':1, '娱乐':2, '家居':3, '房产':4,'教育':5, 
             '时尚':6, '时政':7, '游戏':8, '科技':9, '财经':10}

dfDatas['label'] = dfDatas['label'].map(labelDict)

#map作用可以通过键取字典中的值映射替换
dfDatas.head()



['体育' '娱乐' '家居' '房产' '教育' '时尚' '时政' '游戏' '科技' '财经']


,features,label
0,"[湖人, 新秀, 客场, 酒店, 醉酒, 闹事, 警局, 一游, 错过, 黄蜂, 之战, 新...",1
1,"[魔鬼, 训练场, 奇兵, 邓华德, 值得, 所有人, 尊敬, 华奥, 星空, 上海, 16...",1
2,"[灰熊, 黑八, 合情, 合理, 马刺, 天敌, 实力, 大大, 低估, 新浪, 体育讯, ...",1
3,"[皇帝, 关键球, 26%, 准度, 强于, 韦德, 最后, 一投, 不是, 乱来, 新浪,...",1
4,"[科比, 拒绝接受, 进一步, 检查, 方式, 治疗, 第五场, 一定, 新浪, 体育讯, ...",1


In [333]:
#==5、划分数据集==
#【注意：这里是先划分了数据集又进行了特征词袋的向量化，
#后面对比下先向量化再划分效果是否一样】
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = \
    train_test_split(dfDatas['features'].values,
                     dfDatas['label'].values,
                     test_size=0.30,
                    random_state=0)
#打印看下训练集与测试集数据数量比，划分比例为30%
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(314,)
(135,)
(314,)
(135,)


In [334]:
#==6、训练特征向量化，用CountVectorizer==
#【注意：向量化之前需做件事--拼接训练集特征
#上面训练集需满足CountVectorizer需要的格式，
#即，每篇文章列表不能是词和词逗开的列表元素，需要用空格拼接】
print("==未拼接之前X_train:")
print(X_train[1])

#拼接X_train
X_train_new = []
for item in X_train:
    try:
        #空格拼接分词
        sentence = ' '.join(item)
        #文章存入新训练集列表
        X_train_new.append(sentence)
    except:
        print(item,word_index)#打印有问题的行、词序号
        
print("==拼接后：==\n",X_train_new[1])


==未拼接之前X_train:
['手机', '网游', '封神', 'OL', '英雄', '之血', '秘密', '封神', 'ol', '第九', '服务器', '英雄', '之血', '上周', '火爆', '开启', '截止', '26', '排榜', '寻英', '新服', '狂欢', '活动', '随之', '举行', '万元', 'RMB', '游戏', '少之又少', '素材', '英雄', '之血', '大礼', '一并', '真是', '乐坏了', '玩家', 'RMB', '不用说', '人人', '英雄', '之血', '究竟', '何方', '宝物', '拥有', '装备', '征战', '封神', '神气活现', '无畏', '冲锋', '打造', '属性', '装备', '英雄', '之血', '当然', '不行', '因为', '制造', '高级', '极品', '必备', '生产', '材料', '通过', '生产', '技能', '三下两下', '生产', '材料', '变极装', '目前', '只有', '封神', 'ol', '参加', '新服', '排榜', '活动', '获得', '稀有', '素材', '英雄', '之血', '封神', '勇士', '参与', '活动', '激情', '打榜夺', '宝贝', '物以稀为贵', '拥有', '宝贝', '定能助', '散发出', '与众不同', '耀眼', '光芒', '征战', '游戏', '所向披靡', '排榜', '寻英', '活动', '13', '日至', '26', '新服', '英雄', '之血', '荣誉', '击杀', '等级', '在线', '各项', '活动', '排名', '20', '玩家', '最终', '获得', '新服', 'RMB', '大奖', '并且', '排名', '玩家', '英雄', '之血', '珍贵', '游戏', '稀有', '素材', '首款', '大型', '3G', '神话', '手机', '网游', '封神', 'online', '精致', '细腻', '立体', '画面', '真实', '唯美', '百种', '场景', '海量', '新奇', '剧情', '任务', '独创', '万种', '极品', '换

In [368]:
#向量化X_train_new
from sklearn.feature_extraction.text import CountVectorizer

#实例化向量构造器CountVectorizer
cv = CountVectorizer()
#参数
#max_features关键词降序，=1000,取前1000个,样本文章太短，此处用默认
#参数可以加ngram_range=(1, 2) 两两组合，增加向量组合，提升训练精确度，
#此处先常规处理。后面改下这里对比下得分变化。

print(cv)#打印可看向量构造器全部参数
#向量构造器训练X_train_new得到向量化的X_train_new
#print(X_train_new)
cvFit = cv.fit(X_train_new)
X_train_cvFit = cv.fit_transform(X_train_new)
#cv.fit(X_train_new) #这样fit ，下一步再cv.transform也可，
#cv已携带了上一步结果,不用赋值给变量了
#打印不重复特征
#print(X_train_cvFit.get_feature_names())
#【？这个找不到，可能是因为数据重组的加了列名】
print(X_train_cvFit.toarray())
#向里数组含义：列表中的每一个[]为一篇文章，
#比对每个分词特征是否在不重复特征列表里，函数该特征则为1，没有则为0，
#再依次比对第二篇文章
'''
#如：训练集不重复特征为['bird','dog','cat','fish']
#第一篇文章[0 1 1 1]则意为它不含关键字bird，有dog cat fish各一个
#第一篇文章[0 2 1 1]则意为它不含关键字bird，有2个dog关键词，cat fish各1个
'''

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 5 0 0]
 [0 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]]


"\n#如：训练集不重复特征为['bird','dog','cat','fish']\n#第一篇文章[0 1 1 1]则意为它不含关键字bird，有dog cat fish各一个\n#第一篇文章[0 2 1 1]则意为它不含关键字bird，有2个dog关键词，cat fish各1个\n"

In [364]:

#==7、优化向量，给0，1，2，3表示的向量加权重，
#用TfidfTransformer TF-IDF 词频-逆文档频率算法==
from sklearn.feature_extraction.text import TfidfTransformer

#实例化Tfidf转换类 
tfidf = TfidfTransformer()

#训练上一步的向量结果
X_train_tfidfFit = tfidf.fit_transform(X_train_cvFit)
print("==TF-IDF细化分词重要性权重==:",X_train_tfidfFit.toarray())


==TF-IDF细化分词重要性权重==: [[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.13051009 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.04699613 0.         0.         ... 0.         0.         0.        ]]


In [372]:
#==8、用朴素贝叶斯算法 训练模型分类==
#【原理：贝叶斯通过不断的调整先验概率，不断获得塑造后验概率，
#最后设置阈值来进行判断分类】
#sklearn库直接使用即可，算法已封装
from sklearn.naive_bayes import MultinomialNB
#单词 Multinomial 多项的、多项式 nomial 分类 NB naive bayes缩写

#【机器学习思想在本例中的解释：
#训的过程就是学习训练集的X是y这个分类的特性，提高判断的概率
#验证的过程是用新的数据test集的X和y代入进去，检验符不符合前面模型的训练结果，
#也就是看分类是否正确，
#正确率得分还行的话，这个模型就可以拿去使用了，比如用于分类其它的文章。
#】

#实例化贝叶斯分类器
classifier= MultinomialNB()

#训练上面加权重后的向量训练集
classifier.fit(X_train_tfidfFit,y_train)#使用TF-IDF后的数据
#classifier.fit(X_train_cvFit,y_train)#使用未TF-IDF的数据

#classifier.fit(cv.transform(X_train_new),y_train)
#【问：此处classifier fit后即保留了优化后的训练结果，携带到下面的测试步步骤，
#无需再赋值给y_pred变量?】




MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [373]:
#==9、测试集 特征 向量化 （处理方式同上面训练集向量化）==
#拼接X_train
X_test_new = []
for item_t in X_test:
    try:
        #空格拼接分词
        sentence_t = ' '.join(item_t)
        #文章存入新训练集列表
        X_test_new.append(sentence_t)
    except:
        print(item_t,word_index)#打印有问题的行、词序号
        
print("==拼接后：==\n",X_test_new[1])
#======cv.fit(X_test_new)====
#【这里注意：测试数据无需做TF-IDF处理，用原始数据测试，才能达到验证目的，
#只需在上面转为分词格式，在下面验证的地方转向量即可】


==拼接后：==
 保持 通话 撞车 古天乐 丧命 汽水 组图 新浪 娱乐 近日 翻拍 好莱坞 电影 致命 来电 英皇 电影 保持 通话 正在 热拍 当中 古天乐 听歌 blog 古仔 演技 受到 前所未有 挑战 早前 逆向 飞车 古天乐 最近 拍摄 连人 带车 穿过 近万罐 汽水 翻车 拍摄 过程 的确 艰巨 连场 飞车 场面 古仔 听歌 最终 一跃 两米 载有 近万罐 汽水 货运车 停下 已有 八年 驾车 古仔 再握 方向盘 经历 连环撞 场面 车压 半死 出车 外后 万罐 汽水 阻挡 去路 目眩 不能 站稳 陈木胜 拍摄 工作 一向 严谨 古仔 最终 需要 车底 趴近 小时 才能 完成 该场 初时 古仔 车内 非常 习惯 小时 渐渐 适应 车内 导演 言谈 甚欢 研究 拍摄 细则 导演 笑言派 饭盒 车下 毫无 问题 古仔 经常 专业 跑道 赛车 绝对 信心 经常 吓坏 因为 车速 太快 担心 意外 停工 所以 此场 趴在 车底 已经 非常 安全 绝不 担心 汽水 天空 一幕 异常 震撼 工作人员 之前 整整 三天 时间 汽水 钻洞 汽水 尽量 减轻 重量 轿车 穿过 万个 汽水 场面 更为 震撼 古仔 坦言 感觉 震撼 从来 没想 汽水 罐竟 营造 如此 铺天盖地 场景 戏中 目眩 其实 真的 因为 满地 汽水 罐令 眩晕 感觉 最惨 长期 车底 拍摄 事后 腰酸背痛 除此之外 戏中 货运车 剧组 货柜车 改装 而成 工作人员 更要 整辆 货柜车 喷油 变成 汽水 运输车 花费 接近 二十万 超过 十天 完工 完成 拍摄 导演 所有 空罐 送给 回收 汽水 婆婆 绝无 浪费资源 TUNGSTAR 文并 



In [374]:
#==10、验证模型==
#print(cv.transform(X_test_new))
classifier.score(cv.transform(X_test_new),y_test)#参数：测试集X，y
#得分0.9037037037037037,还可以
#上面n-gram那里改成bi-gram看下分值0.9037037037037037，好像没什么变化，
#汉语词组这种二元组合的情况比英文少
#把上面的tf-Idf部分去掉看下分值0.9259259259259259
#……有意思了，看来，不用tf-idf向量的分值高一些，
#说明不用tf-Idf的过拟合风险高一些,粗分的，分值当然会高。

0.9037037037037037

In [ ]:
#【总结：注意，NLP 文本分类 训练等 也是要 划发数据集，
#训练集训练模型，调参，测试集验证，最后用陌生数据泛化使用。
#本例是有监督学习，有label的比较和分类任务。
#无监督学习，没有label的，也要分训练集测试集，
#训练思想可能会是训练找到高词频的前排关键词，用于做其它任务，做词云，或者情感分析之类的】
#【步骤总结：文本特征提取（汉语分词，去停用词）、文本特征表示（向量化）、归一化处理（预处理）、文本分类】

In [388]:
'''
#==11、svm支持向量机分类==
'''
from sklearn.svm import SVC
from sklearn import metrics

#实例化
clfSvm = SVC(kernel='linear')
#单词kernel核 linear 为线性二维平面SVM,rbf为非线性高维SVM
#低维不可分问题可以用rbf变高维可分，相当于把一组混合的类别提升到另一层面，空间分离开

#训练数据
clfSvmFit = clfSvm.fit(X_train_tfidfFit,y_train)#tf-idf加权重向量
#clfSvmFit = clfSvm.fit(X_train_cvFit,y_train)#CounterVector向量
print("分类器实例对象",clfSvmFit)
#预测
y_pred = clfSvmFit.predict(cv.transform(X_test_new))
print("==预测值y_pred：==\n",y_pred)
#验证
print("==测试集y_test:==\n",y_test)
#召回率
recallScore = metrics.recall_score(y_test,y_pred,average='macro')
#精确度
preciCore = metrics.precision_score(y_test,y_pred,average='macro')
#上面加,average='macro'宏平均，解决下面两行bug
#Target is multiclass but average='binary'. 
#Please choose another average setting.
print("==召回率：==\n",recallScore)#0.9354646854646855
print("==精确度：==\n",preciCore)#0.9375757575757575
#SVM分数比贝叶斯高一点，但两个用的评估方法也不同,SVM肯定是更强大的


分类器实例对象 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
==预测值y_pred：==
 [ 4  2  9  9  6  4  1  9  7  2  9  3  9  5  9  1  4  1  1  7  9  5 10  3
  9  9  9  3  6  6 10  9  8  2  6  6  9  6  9  3  5  6  9  6  9  9  3  4
  9  2  5  9  5  1  3  2  9  1  2  9  4  9  5  2  5  3  3  9 10  9  6  9
  9  4  2  9  5 10  2  4  9 10  4 10  8  5  5  5  7  5  2  2  2  1  2  3
  5  1  7  6  6  8  1  7  1 10  9  4  1  4  9  1  2  9  1  9  8  1  9  7
  9  7  5  2  2  2  8  5  3  9  8  9 10 10  9]
==测试集y_test:==
 [ 4  2  9  9  6  4  1  9  7  2  9  5  9  5  9  1  4  1  1  7  9  5 10  3
  9  9  9  6  6  6 10  9  8  2  6  6  9  6  9  9  5  6  9  6  9  9  3  4
  9  2  5  9  5  1  3  2  9  1  2  9  4  9  7  2  5  3  3  9 10  9  3  9
  9  4  2  9  5 10  2  4  9 10  4 10  8  5  5  5  7  7  2  2  2  1  2  4
  5  1  7  6  6  8  1  7  1 

In [401]:
'''
#==13、LDA生成文档主题==
#本例使用sklearn库完成，其它库也可以，如gensim
'''
from sklearn.decomposition import LatentDirichletAllocation

#实例化主题模型
lda = LatentDirichletAllocation(n_topics=18, 
                                max_iter=50,
                                learning_method='batch')
                                #参数：指定最大主题，迭代次数

lda.fit(X_train_tfidfFit)
themeMatrix = lda.components_
#print("==向量对应汉字:\n",cv.get_feature_names())
print("==主题-词语分布矩阵:\n",themeMatrix)
for topic_idx,topic_line in enumerate(themeMatrix):
    print("==话题top %s :\n" % topic_idx)
    print("==本篇热门话题==:\n",topic_line)
    #【后面这里加下对应的汉字】
    

G:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


==主题-词语分布矩阵:
 [[0.05555558 0.05555556 0.05555556 ... 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.05555556 ... 0.18606553 0.05555556 0.05555556]
 [0.14353537 0.05555556 0.05555556 ... 0.05555556 0.05555556 0.05555556]
 ...
 [0.05555556 0.05555556 0.05555556 ... 0.05555556 0.05555556 0.05555556]
 [0.05555556 0.05555556 0.07356107 ... 0.05555556 0.09125618 0.05555556]
 [0.05555556 0.05555556 0.05555556 ... 0.05555556 0.05555556 0.05555556]]
==话题top 0 :

==本篇热门话题==:
 [0.05555558 0.05555556 0.05555556 ... 0.05555556 0.05555556 0.05555556]
==话题top 1 :

==本篇热门话题==:
 [0.05555556 0.05555556 0.05555556 ... 0.18606553 0.05555556 0.05555556]
==话题top 2 :

==本篇热门话题==:
 [0.14353537 0.05555556 0.05555556 ... 0.05555556 0.05555556 0.05555556]
==话题top 3 :

==本篇热门话题==:
 [0.10195679 0.05555556 0.0555651  ... 0.05555556 0.05555556 0.05555556]
==话题top 4 :

==本篇热门话题==:
 [0.05555556 0.05555556 0.05555556 ... 0.05555556 0.05555556 0.05555556]
==话题top 5 :

==本篇热门话题==:
 [0.05559271 0.05555775 0.0

In [402]:
'''
#==13、将以上每篇主题词做为新特征合并到Xtrain中，再次进行分类==
'''
import numpy as np

themeMatrix.shape#查看主题矩阵维度(18行, 19796列)


(18, 19796)

In [398]:
X_train_tfidfFit.shape#查看X训练集矩阵维度(314行, 19796列)

(314, 19796)

In [400]:
y_train.shape#查看y训练集维度(314行一维,)

(314,)

In [ ]:
#合并矩阵
X_train_newMatrix= np.hstack((themeMatrix,X_train_tfidfFit))
#hstack水平方向合并矩阵，vstack垂直方向合并矩阵
#【报错，待查】

In [423]:
'''
#矩阵合并测试
a=np.matrix([1,2,3])
b=np.matrix([[4,5,6],
  ['a','b','c']])
a.shape#(1,3)
b.shape#(2,3)
d = np.vstack((a,b))
print(d)
d.shape#(3,3)
'''

[['1' '2' '3']
 ['4' '5' '6']
 ['a' 'b' 'c']]


(3, 3)

In [ ]:
'''
#重新训练组合了新特征的X训练集矩阵做分类任务
'''
#训练数据
clfSvmFit2 = clfSvm.fit(X_train_newMatrix,y_train)#加入主题特征矩阵，y训练集

#预测
y_pred2 = clfSvmFit2.predict(cv.transform(X_test_new))

#验证
print("==测试集y_test:==\n",y_test)
#召回率
recallScore2 = metrics.recall_score(y_test,y_pred2,average='macro')
#精确度
preciCore2 = metrics.precision_score(y_test,y_pred2,average='macro')

print("==召回率：==\n",recallScore2)
print("==精确度：==\n",preciCore2)


In [ ]:
'''
1. 朴素贝叶斯的原理

先验概率 p(x) 事先给它的概率，通常先设为50%
后验概率 p(x|y) 通过出现的数据调整了它的概率。
贝叶斯通过不断的调整先验概率，不断获得塑造后验概率
最后设置阈值来进行判断分类

2. 利用朴素贝叶斯模型进行文本分类（见下文）

3. SVM的原理

支持向量机（support vector machine）:它也是用来解决二分类问题，也解决多分类。
和逻辑回归的区别是分开类别的不是一条线，而是一条隔离带（决策边界），
需满足最近的特征点到隔离带中间线的距离最大，也就是隔离带最宽。这时泛化能力最强。
为了提高泛化能力，引入C松弛因子，目的是划线分类时允许忽略个别异常点，这些点可能对决策边界产生巨大影响。
只有决策边界上的点会影响到决策边界的划定，这些点叫支持向量。其它的点无影响。
上面是线性支持向量机，也就是二维空间的。

非线性支持向量机：有些分类不好分，比如两簇特征点是环形围绕的，在二维平面用直线很难分，
这时可引入SVM的核函数，把低维向量空间转为高维，
相当于把其中一个分类提升到另一个层面，在两个分类中间画一个平面分隔，
思想是把低维不可分问题转换为高维可分问题。
在计算量上，利用了数学里的一个巧合，即先算低维空间里的内积（公式中的一步）再映射到高维，
和先映映射再做内积结果一样。
也就是说，计算还是在低维计算，没有增加计算量，高维映射只是一个假设，只是把结果做了高维变换。
所以SVM是非常强大的一个算法，机器学习里很常用。

4. 利用SVM模型进行文本分类（见上文）

5. pLSA、共轭先验分布；LDA主题模型原理：

pLSA模型：
基于概率统计的pLSA模型(probabilistic latentsemantic analysis, 概率隐语义分析)，增加了主题模型，
形成简单的贝叶斯网络，可以使用EM算法学习模型参数。


LDA：
(Latent Dirichlet Allocation)
#Latent潜伏的，Dirichlet狄利克雷高维分布 Allocation分配
文档主题生成模型，
LDA比PLSA更高级，是也称为一个三层贝叶斯概率模型，包含词、主题和文档三层结构。
训练LDA的一大目的就是分析一篇文章的话题分布。
我们认为一篇文章的每个词都是通过“以一定概率选择了某个主题，并从这个主题中以一定概率选择某个词语”这样一个过程得到。
文档到主题服从多项式分布，主题到词服从多项式分布。
是一种非监督机器学习技术，可以用来识别大规模文档集（document collection）或语料库（corpus）中潜藏的主题信息。
它采用了词袋（bag of words）的方法
LDA生成过程：
(1)对每一篇文档，从主题分布中抽取一个主题;
(2)从上述被抽到的主题所对应的单词分布中抽取一个单词;
(3)重复上述过程直至遍历文档中的每一个单词。  
语料库中的每一篇文档与T（通过反复试验等方法事先给定）个主题的一个多项分布 （multinomialdistribution）相对应，将该多项分布记为θ。每个主题又与词汇表（vocabulary）中的V个单词的一个多项分布相对应，将这个多项分布记为φ。
LDA的核心公式：  
p(w|d)=p(w|t)*p(t|d)  
就是以Topic作为中间层，可以通过当前的θd和φt给出了文档d中出现单词w的概率。其中p(t|d)利用θd计算得到，p(w|t)利用φt计算得到。 

共轭先验分布：

两个及以上的对象，互相牵制、控制。 
那在贝叶斯理论里呢，在已知似然函数情况下（已经有样本数据了），
根据先验概率函数求后验概率，
选取什么样的先验分布，会让后验分布与先验分布具有相同的数学形式？
由此引出共轭分布理论。 

6. 使用LDA生成主题特征，在之前特征的基础上加入主题特征进行文本分类（见下文）

'''